<h1><b>Decision Support System for Credit Approval</b></h1>
<h2>Antonio Dottori</h2>
The objective here is to build a classifier that will say if someone will have good credit. 


Importing the libraries we'll need:

In [56]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import impute
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score

Initially, let's read the files containg the data. Here, we have two switches: <br>

training --> if True, it will run for the train set, and if False, it will run for the test set. To use, you set it first as True and goes to Run All. After that, set it to False and Run All again.<br>
finding_best_parameters --> if True, it will run the steps in which we determine the best fits for each model we used. If false, it will run with the values already determined (which were gotten after running this as True.)

In [57]:
credit_training = pd.read_csv("../files/lesson_1/conjunto_de_treinamento.csv", sep=",")
credit_testing = pd.read_csv("../files/lesson_1/conjunto_de_teste.csv", sep=",")

training = True
finding_best_parameters = False

if training:
    credit = credit_training
else:
    credit = credit_testing

For a quick check if it's loaded, let's call the DataFrame's head().

In [58]:
credit.head()

,id_solicitante,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,...,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente
0,1,1,10,presencial,1,M,85,2,0,0,...,N,,0,9.0,1.0,0.0,0.0,600.0,600.0,0
1,2,1,25,internet,1,F,38,1,0,0,...,N,,0,2.0,5.0,NaN,NaN,492.0,492.0,0
2,3,1,20,internet,1,F,37,2,0,0,...,N,,0,NaN,NaN,NaN,NaN,450.0,450.0,1
3,4,1,20,internet,1,M,37,1,1,0,...,Y,54,0,9.0,2.0,NaN,NaN,932.0,932.0,1
4,5,7,1,internet,1,F,51,1,3,0,...,N,,0,9.0,5.0,NaN,NaN,440.0,440.0,1


<p>We can also see the transposed DataFrame to take a look at the columns.</p>

In [59]:
credit.T

,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
id_solicitante,1,2,3,4,5,6,7,8,9,10,...,19991,19992,19993,19994,19995,19996,19997,19998,19999,20000
produto_solicitado,1,1,1,1,7,1,1,1,2,1,...,1,1,1,1,2,1,1,1,1,2
dia_vencimento,10,25,20,20,1,20,15,5,25,10,...,10,10,5,5,5,10,20,10,5,20
forma_envio_solicitacao,presencial,internet,internet,internet,internet,presencial,presencial,internet,internet,presencial,...,presencial,presencial,internet,internet,presencial,presencial,presencial,internet,internet,presencial
tipo_endereco,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
sexo,M,F,F,M,F,M,F,F,F,M,...,F,M,M,F,M,M,F,F,F,F
idade,85,38,37,37,51,21,64,20,39,44,...,52,48,62,18,23,27,26,63,84,53
estado_civil,2,1,2,1,1,1,4,1,2,2,...,4,2,4,1,2,2,2,2,1,1
qtde_dependentes,0,0,0,1,3,1,2,0,2,2,...,0,2,0,0,0,0,1,0,0,0
grau_instrucao,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h2><b>Data Describing</b></h2><Br>
Let's look at some information on our data: count, mean and standart deviation.

In [60]:
credit.describe().T

,count,mean,std,min,25%,50%,75%,max
id_solicitante,20000.0,10000.500000,5773.647028,1.0,5000.75,10000.5,15000.25,20000.0
produto_solicitado,20000.0,1.284350,1.008239,1.0,1.00,1.0,1.00,7.0
dia_vencimento,20000.0,13.147250,6.748507,1.0,10.00,10.0,20.00,25.0
tipo_endereco,20000.0,1.006350,0.079435,1.0,1.00,1.0,1.00,2.0
idade,20000.0,42.352500,14.930177,7.0,31.00,40.0,52.00,106.0
estado_civil,20000.0,2.120850,1.332004,0.0,1.00,2.0,2.00,7.0
qtde_dependentes,20000.0,0.666400,1.236725,0.0,0.00,0.0,1.00,53.0
grau_instrucao,20000.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0
nacionalidade,20000.0,0.961600,0.202305,0.0,1.00,1.0,1.00,2.0
tipo_residencia,19464.0,1.261303,0.883580,0.0,1.00,1.0,1.00,5.0


Let's see the correlation matrix to try and extract some useful information.

In [61]:
credit_correlation = credit.corr(method='pearson')
credit_correlation.style.background_gradient(cmap='coolwarm')

C:\Users\dotto\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\formats\style.py:1264: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(s.to_numpy()) if vmin is None else vmin
C:\Users\dotto\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\formats\style.py:1265: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(s.to_numpy()) if vmax is None else vmax


,id_solicitante,produto_solicitado,dia_vencimento,tipo_endereco,idade,estado_civil,qtde_dependentes,grau_instrucao,nacionalidade,tipo_residencia,meses_na_residencia,possui_email,renda_mensal_regular,renda_extra,possui_cartao_visa,possui_cartao_mastercard,possui_cartao_diners,possui_cartao_amex,possui_outros_cartoes,qtde_contas_bancarias,qtde_contas_bancarias_especiais,valor_patrimonio_pessoal,possui_carro,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente
id_solicitante,1.000000,0.015697,0.008331,0.008912,-0.005783,-0.007921,-0.015013,nan,-0.002403,0.008609,-0.000057,-0.000136,0.000975,0.010474,0.001900,-0.000047,-0.002134,0.004471,-0.002021,0.005294,0.005294,-0.007091,0.000465,0.004115,0.000233,0.005588,-0.011571,-0.023950,0.004528,0.004528,0.017546
produto_solicitado,0.015697,1.000000,0.009396,-0.000695,-0.000132,0.012239,0.018454,nan,0.004016,-0.000131,-0.022069,0.000984,-0.001501,-0.000795,0.000587,0.000511,-0.000123,-0.005108,0.013821,0.020143,0.020143,0.005969,0.021562,-0.002884,-0.012654,0.011132,-0.019514,-0.002462,-0.068252,-0.068252,0.030103
dia_vencimento,0.008331,0.009396,1.000000,0.009915,-0.144137,-0.037564,0.013669,nan,0.010038,0.012612,-0.027111,0.032484,-0.007817,-0.003177,-0.001349,0.010549,-0.010328,-0.000661,0.011630,-0.027109,-0.027109,-0.005361,-0.033565,-0.005244,-0.023262,0.096025,0.004364,0.012895,-0.060951,-0.060951,0.080338
tipo_endereco,0.008912,-0.000695,0.009915,1.000000,-0.020733,-0.006308,0.010367,nan,0.024509,0.016558,-0.011814,-0.000783,0.037032,0.003694,-0.003698,-0.006568,-0.003200,-0.003623,-0.003796,-0.011591,-0.011591,0.084898,-0.005398,-0.001830,0.014834,0.007049,0.004424,0.012147,0.006914,0.006914,-0.004406
idade,-0.005783,-0.000132,-0.144137,-0.020733,1.000000,0.240774,-0.034122,nan,0.010061,-0.107336,0.200798,-0.027884,-0.000676,0.013038,-0.022628,-0.043246,0.003915,-0.002033,0.008280,0.033182,0.033182,0.003181,0.027468,0.019810,-0.022335,-0.231184,-0.031238,-0.065704,-0.003233,-0.003233,-0.120678
estado_civil,-0.007921,0.012239,-0.037564,-0.006308,0.240774,1.000000,0.051277,nan,0.052664,-0.000283,0.052277,-0.027612,0.002221,-0.000172,0.001421,-0.024906,0.003882,-0.007432,0.004406,0.005479,0.005479,0.007007,0.010191,0.002943,-0.004528,-0.070680,-0.084149,0.008368,0.017324,0.017324,-0.031269
qtde_dependentes,-0.015013,0.018454,0.013669,0.010367,-0.034122,0.051277,1.000000,nan,-0.074386,-0.009213,-0.039442,0.013248,0.008208,0.003427,0.024994,0.004207,-0.001340,-0.000288,0.000864,0.020189,0.020189,0.004300,0.016154,0.012308,0.027559,0.024713,0.134472,0.080710,0.039887,0.039887,0.017628
grau_instrucao,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nacionalidade,-0.002403,0.004016,0.010038,0.024509,0.010061,0.052664,-0.074386,nan,1.000000,-0.008975,0.171580,-0.095146,-0.008573,0.005827,-0.044720,-0.048124,0.007599,-0.007790,0.003798,-0.216606,-0.216606,0.009595,-0.239442,0.004344,-0.120699,0.032588,-0.451606,0.113216,0.025991,0.025991,0.000494
tipo_residencia,0.008609,-0.000131,0.012612,0.016558,-0.107336,-0.000283,-0.009213,nan,-0.008975,1.000000,-0.093792,0.005118,-0.005948,-0.001789,-0.006506,-0.013981,-0.003390,-0.007001,0.005135,-0.011661,-0.011661,-0.009362,-0.003904,0.010398,0.037838,-0.010736,0.017603,0.026397,-0.007120,-0.007120,0.018223


<h2><b>Data Analysis and Cleaning</b></h2>

In this DataFrame there are columns that are not so useful for our analysis. For this reason, let's remove them.

id --> just an index value; <br>
grau_instrucao --> this column is filled by only one value; <br>
possui_telefone_celular --> only one value on the entire column  <br>
estado_onde_trabalha --> too many blanks <br>
codigo_area_telefone_trabalho --> too many blanks  <br>
meses_no_trabalho --> biggest part is equal to zero  <br>
profissao_companheiro --> too many blanks <br>
grau_instrucao_companheiro --> too many blanks<br>

In [62]:
columns_to_remove = [
    "id_solicitante",
    "grau_instrucao",
    "possui_telefone_celular",
    "estado_onde_trabalha",
    "codigo_area_telefone_trabalho",
    "meses_no_trabalho",
    "profissao_companheiro",
    "grau_instrucao_companheiro"
]

columns_to_keep = [column for column in credit.columns if column not in columns_to_remove]
credit = credit[columns_to_keep]

Now, we'll split the columns into their types. They can be categorical variables, i.e., those that, even though they may be numbers, they are mentions to a category of some kind, or numerical variables. <br>

<h2><b>Categorical Variables</b></h2>

<br>Filtering the "object type" columns gives us a first filter for this classification, but we can also select manually some other columns that we find suitable. Another concept we should have in mind is that the categorical variables may be nominal (i.e., they are in no way related in an order of magnitude, they just represent a category) or ordinal (i.e., they have an order of importance inherited.).


In [63]:
credit.columns[credit.dtypes==object]

Index(['forma_envio_solicitacao', 'sexo', 'estado_onde_nasceu',
       'estado_onde_reside', 'possui_telefone_residencial',
       'codigo_area_telefone_residencial', 'vinculo_formal_com_empresa',
       'possui_telefone_trabalho'],
      dtype='object')

<h3><b>Nominal Variables</b></h3><br>As stated, we will use the filtered columns from the previous step, and add some more that fit into this category.

In [64]:
nominal_categorical_columns = [
    'forma_envio_solicitacao',
    'sexo', 
    'estado_onde_nasceu', 
    'estado_onde_reside', 
    'possui_telefone_residencial', 
    'codigo_area_telefone_residencial', 
    'vinculo_formal_com_empresa', 
    'possui_telefone_trabalho', 
    'produto_solicitado', 
    'dia_vencimento', 
    'tipo_endereco', 
    'estado_civil', 
    'nacionalidade', 
    'tipo_residencia',
    'possui_cartao_visa', 
    'possui_cartao_mastercard', 
    'possui_cartao_diners', 
    'possui_cartao_amex', 
    'possui_outros_cartoes', 
    'possui_email', 
    'possui_carro', 
    'profissao', 
    'ocupacao', 
    'local_onde_reside', 
    'local_onde_trabalha',
]

Let's look at the cardinality, and we will drop the columns that have a great variety, because, as categorical values, they will have a lot of noise attached to them.

In [65]:
credit[nominal_categorical_columns].nunique().sort_values(ascending=False)

local_onde_trabalha                 743
local_onde_reside                   743
codigo_area_telefone_residencial     81
estado_onde_nasceu                   28
estado_onde_reside                   27
profissao                            18
estado_civil                          8
ocupacao                              6
tipo_residencia                       6
dia_vencimento                        6
sexo                                  4
forma_envio_solicitacao               3
produto_solicitado                    3
nacionalidade                         3
tipo_endereco                         2
possui_cartao_visa                    2
possui_cartao_mastercard              2
possui_cartao_diners                  2
possui_cartao_amex                    2
possui_outros_cartoes                 2
possui_email                          2
possui_carro                          2
possui_telefone_trabalho              2
vinculo_formal_com_empresa            2
possui_telefone_residencial           2


In [66]:
nominal_categorical_columns_to_delete =[
    "local_onde_trabalha",
    "local_onde_reside",
    "codigo_area_telefone_residencial",
    "estado_onde_nasceu",
    "estado_onde_reside",
]

columns_to_keep = [column for column in credit.columns if column not in nominal_categorical_columns_to_delete]
credit = credit[columns_to_keep]

One last thing we need to do is take care of the NaN values in these columns. Let's split the columns in two groups: those that have NaN values (or empties, that we'll replace to NaN), and those that don't have NaN values.

In [67]:
remaining_nominal_categorical_columns = [column for column in nominal_categorical_columns if column not in nominal_categorical_columns_to_delete]

credit = credit.replace("", np.nan)
credit = credit.replace(" ", np.nan)

remaining_nominal_categorical_columns_with_nan = [column for column in remaining_nominal_categorical_columns if credit[column].isnull().any()]
remaining_nominal_categorical_columns_without_nan = [column for column in remaining_nominal_categorical_columns if column not in remaining_nominal_categorical_columns_with_nan]

In [68]:
remaining_nominal_categorical_columns_with_nan

['sexo', 'tipo_residencia', 'profissao', 'ocupacao']

<h4>"sexo" column</h4>

In [69]:
credit["sexo"].unique()

array(['M', 'F', 'N', nan], dtype=object)

In [70]:
credit["sexo"] = credit["sexo"].map({"M":0, "F":1, "N": np.nan})

On these columns that have NaN values, we will transform them on the average of the other numbers.

In [71]:
imputer = impute.SimpleImputer(missing_values=np.nan, strategy="mean")
credit[remaining_nominal_categorical_columns_with_nan] = imputer.fit_transform(credit[remaining_nominal_categorical_columns_with_nan])

Among the columns that don't have NaN values, there are some that have only "yes" and "no" values. Let's turn these into True or False. Finally, for the other categorical columns that were not handled yet, we'll perform an ONE-HOT-ENCODING.


In [72]:
yes_or_no_categorical_columns = [
    "possui_telefone_residencial",
    "vinculo_formal_com_empresa",
    "possui_telefone_trabalho",
]

label_binarizer = preprocessing.LabelBinarizer()

for column in yes_or_no_categorical_columns:
    credit[column] = label_binarizer.fit_transform(credit[column])


In [73]:
columns_to_one_hot_encode = [column for column in remaining_nominal_categorical_columns_without_nan if column not in yes_or_no_categorical_columns] + remaining_nominal_categorical_columns_with_nan

credit = pd.get_dummies(credit, columns=columns_to_one_hot_encode)

<h3><b>Numerical</b></h3><br>All the other columns that were not mentioned yet are numerical columns.


In [74]:
numerical_categorical_columns = [
  'idade',
  'qtde_dependentes', 
  'meses_na_residencia', 
  'renda_mensal_regular',
  'renda_extra', 
  'qtde_contas_bancarias', 
  'qtde_contas_bancarias_especiais',
  'valor_patrimonio_pessoal'
]

Checking for any NaN values here and filling them.

In [75]:
credit[numerical_categorical_columns].isna().any()

idade                              False
qtde_dependentes                   False
meses_na_residencia                 True
renda_mensal_regular               False
renda_extra                        False
qtde_contas_bancarias              False
qtde_contas_bancarias_especiais    False
valor_patrimonio_pessoal           False
dtype: bool

In [76]:
imputer_mean = impute.SimpleImputer(missing_values=np.nan, strategy="mean")
credit[['meses_na_residencia']] = imputer_mean.fit_transform(credit[['meses_na_residencia']])

And for a quick check if there are any NaN values:

In [77]:
credit.isna().any().sum()

0

No more NaN's. Great!

<h1><b>Training</b></h1><br>
Let's go the training! If we're training our model, we need to shuffle the DataFrame.

In [78]:
if training:
    shuffled_credit = credit.sample(frac=1, random_state=12345)


In [79]:
if training:
    x = shuffled_credit.iloc[:, shuffled_credit.columns!='inadimplente'].values
    y = shuffled_credit.iloc[:, shuffled_credit.columns=='inadimplente'].values
else:
    x = credit.iloc[:, credit.columns!='inadimplente'].values
    


<p>And now, if we're training, we'll divide the DataFrame in a training and a test set. We need to divide into two reasonable sizes, so there is a training set big enough so that the Machine will be able to be  well trained. Also, we need a good test set size so we can see clearly if it's working fine on the predictions. A good ratio should be training:test as 80:20.</p>

In [80]:
if training:
    num_lines = shuffled_credit.shape[0]
    training_size = round(0.8*num_lines)

In [81]:
if training:
    x_training = x[:training_size,:-1]
    y_training = y[:training_size,-1].ravel()

    x_test = x[training_size:,:-1]
    y_test = y[training_size:,-1].ravel()

else:
    x_test = x

Let's begin the training for different regressors. For each one of them, we'll train the set for various input parameters to see which one will give us the smallest error.

<h1><b>CatBoost</b></h1>

In [82]:
if training and finding_best_parameters:
    catBst = CatBoostClassifier()
    catBst.fit(x_training, y_training)

    results = []
    for iterations in range(20, 220, 20):
        for depth in range(2, 14, 2):
            catBst = CatBoostClassifier(depth=depth, iterations=iterations, learning_rate=0.1, verbose=False)
            catBst.fit(x_training, y_training)
            y_test_answer = catBst.predict(x_test)
            results.append((iterations, depth, accuracy_score(y_test, y_test_answer)))
    best_choose = sorted(results, key=lambda i: i[-1])[-1]
    print (best_choose)

#best choose -> (120, 4, 0.59125)

In [83]:
if training:
    catBst = CatBoostClassifier(depth=4, iterations=120, learning_rate=0.1, verbose=False)
    catBst.fit(x_training, y_training)

y_test_answer_cb = y_test_answer = catBst.predict(x_test)

In [84]:
if training:
    print('Acuracia:', accuracy_score(y_test, y_test_answer_cb))

Acuracia: 0.59125


<h1><b>KNN</b></h1>

In [85]:
if training and finding_best_parameters:
    results = []

    for k in range(1,60,2):
        classifier = KNeighborsClassifier(n_neighbors=k, weights='uniform')
        classifier = classifier.fit(x_training, y_training)
        
        y_training_answer = classifier.predict(x_training)
        y_test_answer = classifier.predict(x_test)

        results.append((iterations, depth, accuracy_score(y_test, y_test_answer)))
        best_choose = sorted(results, key=lambda i: i[-1])[-1]
        print (k, (iterations, depth, accuracy_score(y_test, y_test_answer)))


In [86]:
if training:
    classifier = KNeighborsClassifier(n_neighbors=53, weights='uniform')
    classifier = classifier.fit(x_training, y_training)
    
y_test_answer_knn = classifier.predict(x_test)

In [87]:
if training:
    print('Acuracia:', accuracy_score(y_test, y_test_answer_knn))

Acuracia: 0.5415


The accuracy we got here is smaller than the CatBooster's.

<h2><b>Cross Validation</b></h2><br>

Finally, let's perform the Cross Validation for the best performing model, which was CatBooster.

In [88]:
if training:
    scores = cross_val_score(
        catBst,
        x,
        y.ravel(),
        cv=8,
    )

    print (
        'scores =',scores,
        'acurácia média = %6.1f' % (100*sum(scores)/8)
        )

scores = [0.5792 0.6016 0.6072 0.5836 0.5836 0.6008 0.588  0.592 ] acurácia média =   59.2


<h2>Exporting the Results</h2><br>
We got the best result from the CatBoost Classifier, so that's the one we will export.
And with this function we will export our test answers in the kaggle format.

In [89]:
if not training:
    pd.DataFrame(
    {
        'id_solicitante': np.arange(20001, 25001),
        'inadimplente': y_test_answer_cb,
    }
).to_csv('teste_kaggle.csv', index=False)